In [1]:
val reportFolder = "reports"
val executionRounds = 10

In [2]:
import java.io.File
import java.util.Properties

val properties = Properties()
File("gradle.properties").bufferedReader().use {
    properties.load(it)
}

val executedTask = properties.getProperty("scenario.task") ?: "assemble"

val projectPath = properties.getProperty("project.path") ?: properties.getProperty("project.git.directory")

val projectName: String = projectPath?.substringAfterLast("\\")?.substringAfterLast("/")?.removeSuffix(".git") ?: "User"


In [3]:
enum class CompilerMetrics(val readableName: String) {
    GRADLE_TASK("Total Gradle task time"),

    COMPILATION_ROUND("Sources compilation round"),
    
    //compiler metrics
    COMPILER_INITIALIZATION("Compiler initialization time"),
    CODE_ANALYSIS("Compiler code analysis"),
    IR_TRANSLATION("Compiler IR translation"),
    IR_LOWERING("Compiler IR lowering"),
    IR_GENERATION("Compiler IR generation"),
    ;
}

In [4]:
%use dataframe

In [5]:
import java.io.File
import org.jetbrains.kotlinx.dataframe.DataFrame

//Json report print time in milliseconds 
fun readJsonReportFile(file: File): Map<CompilerMetrics, Long?>? {
    if (file.extension != "json" || file.readText().isBlank())
        return null
    val report = DataFrame.read(file)

    if ((report.get("startParameters").get("tasks")[0] as List<String>).get(0) != executedTask)
        return null

    val timeMetrics = report.get("aggregatedMetrics").get("buildTimes").get("buildTimesNs")[0] as DataRow<*>

    return CompilerMetrics.values().associate {
        val value = timeMetrics.getOrNull(it.name)?.let {
            when (it) {
                is Int -> it.toLong()
                is Long -> it
                is String -> it.toLong()
                else -> throw IllegalStateException("Unknown type ${it.javaClass} to convert to Long")
            }
        }
        Pair(it, value)
    }
}

In [7]:
import kotlin.streams.toList

//Text report print time in seconds with 2 decemal digits, so there is an error during conversion to nanoSeconds
fun readTextReportFile(file: File): Map<CompilerMetrics, Long?>? {
    val isValidReportFile = file.extension == "txt" && file.readText().isNotEmpty()  && file.readText().contains("tasks = [$executedTask]")
    if (!isValidReportFile) return null
    
    val aggregatedTimeMetrics = file.bufferedReader().use {
        it.lines().dropWhile { !it.startsWith("Time metrics:") }.takeWhile { it.isNotBlank() }.toList()
    }

    return CompilerMetrics.values().associate { metric ->
        val metricLine = aggregatedTimeMetrics.firstOrNull { it.trim().startsWith(metric.readableName) }
        val value = metricLine?.substringAfter(": ")?.substringBefore(" ")?.replace(",", "")?.toDouble()
        Pair(metric, value?.times(1_000_000_000)?.toLong())
    }
}


# Read compiler metrics


In [8]:
%use kandy

In [9]:
import org.jetbrains.kotlinx.dataframe.api.toDataFrame
import java.nio.file.Files
import kotlin.io.path.Path

fun readMetrics(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>?
) = directory.listFiles()
    .sortedByDescending(File::lastModified)
    .mapNotNull { readReportFile(it) }
    .take(executionRounds)
    .toList()

fun collectMeanValue(
    metrics: List<Map<CompilerMetrics, Long?>>,
): Map<String, Any?> {
    val rows = CompilerMetrics.values().map { compilerMetric ->
        val values = metrics.map { it[compilerMetric] }.filterNotNull()
        Pair(compilerMetric.name, values)
    }.filter { 
        val allValuesPresent = (it.second.size == executionRounds)
        if (!allValuesPresent) {
            println("Unable to calculate mean value for ${it.first}, only ${it.second.size} runs are found")
        }
        allValuesPresent
    }.toTypedArray()

    val meanValue = dataFrameOf(*rows).mean()

    return CompilerMetrics.values().associate {
        Pair(it.name, meanValue.getOrNull(it.name))
    }
} 


# Collect data into dataframe

In [11]:
val kotlinVersionColumn = "kotlinVersion"
val buildScenarioColumn = "buildScenario"

val columnNames = CompilerMetrics.values().map { it.name }.toMutableList()
columnNames.add(kotlinVersionColumn)
columnNames.add(buildScenarioColumn)

fun collectResultsIntoDataFrame(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>?,
): AnyFrame {
    val rows = directory.listFiles().flatMap { versionDir ->
        versionDir.listFiles().map { scenarioDir ->
            val buildMetrics = readMetrics(scenarioDir, readReportFile)
            //rename scenarios for graphics only
            val scenario = when (scenarioDir.name) {
                "clean_build" -> "Clean build"
                "incremental_abi_build" -> "Incremental build for ABI changes"
                "incremental_non_abi_build" -> "Incremental build for non-ABI changes"
                else -> scenarioDir.name
            }

            println("\n Validate \'$scenario\' scenario for ${versionDir.name} kotlin version")
            val row = collectMeanValue(buildMetrics).toMutableMap()
            row.put(kotlinVersionColumn, versionDir.name)
            row.put(buildScenarioColumn, scenario)
            row
        }
    }

    val db = columnNames.map { compilerMetric -> Pair(compilerMetric, rows.map { it[compilerMetric] })}.toTypedArray()
    return dataFrameOf(*db)
}

In [12]:
import java.util.concurrent.TimeUnit
import kotlin.time.DurationUnit
import kotlin.time.toDuration

enum class TimeTransformation(val printTime: String, val transformation: (Double?) -> Double?) {
    NANOSECONDS("nanosaconds", { it }),
    MILLISECONDS("milliseconds", { 
        it?.toDuration(DurationUnit.NANOSECONDS)?.toDouble(DurationUnit.MILLISECONDS) 
    }),
    SECONDS("seconds", {
        it?.toDuration(DurationUnit.NANOSECONDS)?.toDouble(DurationUnit.SECONDS) 
    })
    ;
}

fun getTimeTranformationForColumn(df: DataFrame<Any?>, column: String): TimeTransformation {
    val meanValue = df.mean(column, skipNA = true)
    val ratio = 1_000
    return when {
        meanValue < 10 * ratio -> TimeTransformation.NANOSECONDS
        meanValue < 10 * ratio * ratio -> TimeTransformation.MILLISECONDS
        else -> TimeTransformation.SECONDS
    }
}

In [13]:
var df = collectResultsIntoDataFrame(File(reportFolder), ::readTextReportFile)
    .sortBy(kotlinVersionColumn)
val timeTransformation = getTimeTranformationForColumn(df, CompilerMetrics.COMPILATION_ROUND.name)

 CompilerMetrics.values().forEach { 
     df = df.update(it.name).with { timeTransformation.transformation(it as Double) }
 }

val dataFrame = df
val scenarios = dataFrame[buildScenarioColumn]


 Validate 'Incremental build for ABI changes' scenario for 1.9.23 kotlin version

 Validate 'Incremental build for non-ABI changes' scenario for 1.9.23 kotlin version

 Validate 'Clean build' scenario for 1.9.23 kotlin version

 Validate 'Incremental build for ABI changes' scenario for 2.0.0-RC1 kotlin version

 Validate 'Incremental build for non-ABI changes' scenario for 2.0.0-RC1 kotlin version

 Validate 'Clean build' scenario for 2.0.0-RC1 kotlin version


In [14]:
val plots = scenarios.distinct().toList().map { scenario ->
    dataFrame.filter { buildScenario == scenario }.plot {
        layout {
            title = "$projectName project"
            subtitle = "$scenario"

            theme() {
                legend.position = LegendPosition.Top
            }
        }
        bars {
            x(kotlinVersionColumn) {
                axis.name = "Gradle compilation time"
            }
            y(CompilerMetrics.COMPILATION_ROUND.name) {
                axis.name = "Time (${timeTransformation.printTime})"
            }
        }
        
    }
}
plotGrid(plots, 1)

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UGgUcZ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"layout":{
"name":"grid",
"ncol":1,
"nrow":3,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"Exposed project",
"subtitle":"Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[0.184,0.138]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle compilation time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
},{
"ggtitle":{
"text":"Exposed project",
"subtitle":"Incremental build for non-ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[0.144,0.124]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle compilation time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
},{
"ggtitle":{
"text":"Exposed project",
"subtitle":"Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[11.502,5.666]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle compilation time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
}],
"kind":"subplots"
};
 var plotContainer = document.getElementById("UGgUcZ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00
 
 
 
 
 
 
 0.05
 
 
 
 
 
 
 0.10
 
 
 
 
 
 
 0.15
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Incremental build for ABI changes
 
 
 
 
 Time (seconds)
 
 
 
 
 Gradle compilation time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00
 
 
 
 
 
 
 0.05
 
 
 
 
 
 
 0.10
 
 
 
 
 
 
 0.15
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Incremental build for non-ABI changes
 
 
 
 
 Time (seconds)
 
 
 
 
 Gradle compilation time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Clean build
 
 
 
 
 Time (seconds)
 
 
 
 
 Gradle compilation time

In [15]:
val incrementalScenatiosLabels = listOf(Pair("Incremental build for ABI changes", "ABI changes"), Pair("Incremental build for non-ABI changes", "non-ABI changes"))

In [16]:
fun gradleBuildTime(scenarios: List<String>, name: String, scenarioBreaksLabeled: List<Pair<String, String>>? = null) =
    dataFrame.filter { buildScenario in scenarios }
        .plot {
            layout {
                title = "$projectName project - $name"
                theme() {
                    legend.position = LegendPosition.Top
                }
            }
            bars {
                x(buildScenarioColumn) {
                    axis.name = "Gradle build time"
                    scenarioBreaksLabeled?.let {
                        axis.breaksLabeled(it.map { it.first }, it.map { it.second })
                    }
                }
                y(GRADLE_TASK.name) {
                    axis.name = "Time (${timeTransformation.printTime})"
                }
                fillColor(kotlinVersionColumn) {
                    legend.name = ""
                }
            }
        }

In [17]:
gradleBuildTime(listOf("Clean build"), "Clean build scenario")

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="BXz5nx"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project - Clean build scenario"
},
"mapping":{
},
"data":{
"GRADLE_TASK":[15.162,8.191],
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"buildScenario":["Clean build","Clean build"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle build time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"GRADLE_TASK",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("BXz5nx");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Clean build
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 2
 
 
 
 
 
 
 4
 
 
 
 
 
 
 6
 
 
 
 
 
 
 8
 
 
 
 
 
 
 10
 
 
 
 
 
 
 12
 
 
 
 
 
 
 14
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project - Clean build scenario
 
 
 
 
 Time (seconds)
 
 
 
 
 Gradle build time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [18]:
gradleBuildTime(listOf("Incremental build for ABI changes", "Incremental build for non-ABI changes"), "Incremental build scenarios", incrementalScenatiosLabels)


<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vI996U"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project - Incremental build scenarios"
},
"mapping":{
},
"data":{
"GRADLE_TASK":[1.158,0.631,0.905,0.505],
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"buildScenario":["Incremental build for ABI changes","Incremental build for non-ABI changes","Incremental build for ABI changes","Incremental build for non-ABI changes"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"breaks":["Incremental build for ABI changes","Incremental build for non-ABI changes"],
"name":"Gradle build time",
"labels":["ABI changes","non-ABI changes"]
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"GRADLE_TASK",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("vI996U");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ABI changes
 
 
 
 
 
 
 
 
 non-ABI changes
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 0.2
 
 
 
 
 
 
 0.4
 
 
 
 
 
 
 0.6
 
 
 
 
 
 
 0.8
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.2
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project - Incremental build scenarios
 
 
 
 
 Time (seconds)
 
 
 
 
 Gradle build time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [19]:
fun kotlinCompilationTime(
    scenarios: List<String>,
    name: String,
    scenarioBreaksLabeled: List<Pair<String, String>>? = null
) = dataFrame.filter { buildScenario in scenarios }
    .plot {
        layout {
            title = "$projectName project - $name"
            theme() {
                legend.position = LegendPosition.Top
            }
        }
        bars {
            x(buildScenarioColumn) {
                axis.name = "Kotlin compilation time"
                scenarioBreaksLabeled?.let {
                    axis.breaksLabeled(it.map { it.first }, it.map { it.second })
                }
            }
            y(COMPILATION_ROUND.name) {
                axis.name = "Time (${timeTransformation.printTime})"
            }
            fillColor(kotlinVersionColumn) {
                legend.name = ""
            }
        }
    }
    


In [20]:
kotlinCompilationTime(listOf("Clean build"), "Clean build scenario")


<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="TFslM3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project - Clean build scenario"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"buildScenario":["Clean build","Clean build"],
"COMPILATION_ROUND":[11.502,5.666]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Kotlin compilation time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"COMPILATION_ROUND",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("TFslM3");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Clean build
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 2
 
 
 
 
 
 
 4
 
 
 
 
 
 
 6
 
 
 
 
 
 
 8
 
 
 
 
 
 
 10
 
 
 
 
 
 
 12
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project - Clean build scenario
 
 
 
 
 Time (seconds)
 
 
 
 
 Kotlin compilation time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [21]:
kotlinCompilationTime(listOf("Incremental build for ABI changes", "Incremental build for non-ABI changes"), "Incremental build scenarios", incrementalScenatiosLabels)

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="mqKtaF"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project - Incremental build scenarios"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"buildScenario":["Incremental build for ABI changes","Incremental build for non-ABI changes","Incremental build for ABI changes","Incremental build for non-ABI changes"],
"COMPILATION_ROUND":[0.184,0.144,0.138,0.124]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"breaks":["Incremental build for ABI changes","Incremental build for non-ABI changes"],
"name":"Kotlin compilation time",
"labels":["ABI changes","non-ABI changes"]
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"COMPILATION_ROUND",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("mqKtaF");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ABI changes
 
 
 
 
 
 
 
 
 non-ABI changes
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00
 
 
 
 
 
 
 0.05
 
 
 
 
 
 
 0.10
 
 
 
 
 
 
 0.15
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project - Incremental build scenarios
 
 
 
 
 Time (seconds)
 
 
 
 
 Kotlin compilation time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

# Compare Gradle and Compiler time

In [22]:

fun getPlotForGradleCompilerTime(scenario: String) = dataFrame.filter { it["buildScenario"] == scenario }
        .gather { GRADLE_TASK.name and COMPILATION_ROUND.name }.into("metric", "time_ns")
        .update("metric").with {
            when (it) {
                "GRADLE_TASK"  -> "Gradle time"
                "COMPILATION_ROUND" -> "Kotlin Compiler time"
                else -> throw IllegalStateException("Unknown metrics: $it")
            }
        }
        .sortBy("time_ns")
        .plot {
            layout {
                title = "$projectName project"
                subtitle = scenario
                caption = "Gradle compilation breakdown"
                theme() {
                    legend.position = LegendPosition.Top
                }
            }
            barsH {
                y(kotlinVersion) {
                    axis.name = "Kotin version"
                }
                x("time_ns"<Double>()) {
                    axis.name = "Time (${timeTransformation.printTime})"
                }
                fillColor("metric") {
                    legend.name = "Build time"
                }
                position = Position.identity()
            }
        }




In [23]:
val versionByScenario =
    scenarios.distinct().toList().associate { name -> Pair(name as String, getPlotForGradleCompilerTime(name)) }

In [24]:
versionByScenario.get("Clean build")

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3L6Tsj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project",
"subtitle":"Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["2.0.0-RC1","2.0.0-RC1","1.9.23","1.9.23"],
"metric":["Kotlin Compiler time","Gradle time","Kotlin Compiler time","Gradle time"],
"time_ns":[5.666,8.191,11.502,15.162]
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Build time"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"identity",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Gradle compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("3L6Tsj");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 6
 
 
 
 
 
 
 
 
 8
 
 
 
 
 
 
 
 
 10
 
 
 
 
 
 
 
 
 12
 
 
 
 
 
 
 
 
 14
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Clean build
 
 
 
 
 Kotin version
 
 
 
 
 Time (seconds)
 
 
 
 
 
 
 
 
 Build time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time
 
 
 
 
 
 
 
 
 Gradle compilation breakdown

In [25]:
versionByScenario.get("Incremental build for non-ABI changes")

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MItyQH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project",
"subtitle":"Incremental build for non-ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["2.0.0-RC1","1.9.23","2.0.0-RC1","1.9.23"],
"metric":["Kotlin Compiler time","Kotlin Compiler time","Gradle time","Gradle time"],
"time_ns":[0.124,0.144,0.505,0.631]
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Build time"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"identity",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Gradle compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("MItyQH");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.1
 
 
 
 
 
 
 
 
 0.2
 
 
 
 
 
 
 
 
 0.3
 
 
 
 
 
 
 
 
 0.4
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 0.6
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Incremental build for non-ABI changes
 
 
 
 
 Kotin version
 
 
 
 
 Time (seconds)
 
 
 
 
 
 
 
 
 Build time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time
 
 
 
 
 
 
 
 
 Gradle compilation breakdown

In [26]:
versionByScenario.get("Incremental build for ABI changes")

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="JDWHkV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project",
"subtitle":"Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["2.0.0-RC1","1.9.23","2.0.0-RC1","1.9.23"],
"metric":["Kotlin Compiler time","Kotlin Compiler time","Gradle time","Gradle time"],
"time_ns":[0.138,0.184,0.905,1.158]
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Build time"
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"identity",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Gradle compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("JDWHkV");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.2
 
 
 
 
 
 
 
 
 0.4
 
 
 
 
 
 
 
 
 0.6
 
 
 
 
 
 
 
 
 0.8
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.2
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Incremental build for ABI changes
 
 
 
 
 Kotin version
 
 
 
 
 Time (seconds)
 
 
 
 
 
 
 
 
 Build time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time
 
 
 
 
 
 
 
 
 Gradle compilation breakdown

# Collect Compiler metrics

In [27]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalTransform


val metricsList = dataFrame
    .gather { COMPILER_INITIALIZATION.name and CODE_ANALYSIS.name and IR_TRANSLATION.name and IR_LOWERING.name and IR_GENERATION.name }.into("metric", "time_ns")


fun getPlotForCompilerMetrics(scenario: String) = metricsList.filter { it["buildScenario"] == scenario }
    .plot {
    layout {
        title = "$projectName project"
        subtitle = scenario
        caption = "Kotlin compilation breakdown"
        theme() {
            legend.position = LegendPosition.Top
        }
        size = Pair(1000, 300)
    }
    barsH {
        y("kotlinVersion"<String>()) {
            axis.name = "Kotlin version"
        }
        x("time_ns"<Double>()) {
            axis.name = "Time (${timeTransformation.printTime})"
        }
        fillColor("metric"<String>()) {
            legend.name = ""
        }
        position = Position.stack()
        
    }
}


In [28]:
val compilerMetrics = scenarios.distinct().toList().associate { Pair(it as String, getPlotForCompilerMetrics(it)) }

In [29]:
compilerMetrics.get("Clean build") 


<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="83fRk0"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project",
"subtitle":"Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[0.662,7.443,0.712,1.117,1.519,0.539,2.837,0.52,0.748,0.975]
},
"ggsize":{
"width":1000.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Kotlin compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("83fRk0");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 1
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 3
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 5
 
 
 
 
 
 
 
 
 6
 
 
 
 
 
 
 
 
 7
 
 
 
 
 
 
 
 
 8
 
 
 
 
 
 
 
 
 9
 
 
 
 
 
 
 
 
 10
 
 
 
 
 
 
 
 
 11
 
 
 
 
 
 
 
 
 12
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Clean build
 
 
 
 
 Kotlin version
 
 
 
 
 Time (seconds)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 COMPILER_INITIALIZATION
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CODE_ANALYSIS
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_TRANSLATION
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_LOWERING
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_GENERATION
 
 
 
 
 
 
 
 
 Kotlin compilation breakdown

In [30]:
compilerMetrics.get("Incremental build for non-ABI changes") 

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="4geSKk"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project",
"subtitle":"Incremental build for non-ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[0.04,0.058,0.017,0.012,0.016,0.039,0.052,0.012,0.013,0.01]
},
"ggsize":{
"width":1000.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Kotlin compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("4geSKk");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00
 
 
 
 
 
 
 
 
 0.01
 
 
 
 
 
 
 
 
 0.02
 
 
 
 
 
 
 
 
 0.03
 
 
 
 
 
 
 
 
 0.04
 
 
 
 
 
 
 
 
 0.05
 
 
 
 
 
 
 
 
 0.06
 
 
 
 
 
 
 
 
 0.07
 
 
 
 
 
 
 
 
 0.08
 
 
 
 
 
 
 
 
 0.09
 
 
 
 
 
 
 
 
 0.10
 
 
 
 
 
 
 
 
 0.11
 
 
 
 
 
 
 
 
 0.12
 
 
 
 
 
 
 
 
 0.13
 
 
 
 
 
 
 
 
 0.14
 
 
 
 
 
 
 
 
 0.15
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Incremental build for non-ABI changes
 
 
 
 
 Kotlin version
 
 
 
 
 Time (seconds)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 COMPILER_INITIALIZATION
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CODE_ANALYSIS
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_TRANSLATION
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_LOWERING
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_GENERATION
 
 
 
 
 
 
 
 
 Kotlin compilation breakdown

In [31]:
compilerMetrics.get("Incremental build for ABI changes") 

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bjffW3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Exposed project",
"subtitle":"Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[0.044,0.089,0.017,0.016,0.017,0.025,0.063,0.019,0.015,0.013]
},
"ggsize":{
"width":1000.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Kotlin compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("bjffW3");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00
 
 
 
 
 
 
 
 
 0.02
 
 
 
 
 
 
 
 
 0.04
 
 
 
 
 
 
 
 
 0.06
 
 
 
 
 
 
 
 
 0.08
 
 
 
 
 
 
 
 
 0.10
 
 
 
 
 
 
 
 
 0.12
 
 
 
 
 
 
 
 
 0.14
 
 
 
 
 
 
 
 
 0.16
 
 
 
 
 
 
 
 
 0.18
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23
 
 
 
 
 
 
 2.0.0-RC1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Exposed project
 
 
 
 
 Incremental build for ABI changes
 
 
 
 
 Kotlin version
 
 
 
 
 Time (seconds)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 COMPILER_INITIALIZATION
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CODE_ANALYSIS
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_TRANSLATION
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_LOWERING
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 IR_GENERATION
 
 
 
 
 
 
 
 
 Kotlin compilation breakdown